# Part I. ETL Pipeline for Pre-Processing the Files

## Pre-process the files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking our current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/saswata/personal/udacity nanodegree data engineer/project_cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. 

### CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance local machine on (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries , so we will create query specific tables

## Query 1: 

### Let us find the artist, song title and song's length in the music app history that was heard during 
### sessionId = 338, and itemInSession = 4

In [8]:
query1 = "select artist,song, length from T_MUSIC where session_id=338 and item_in_session = 4"

### In this query, we will select columns artist, song title and length of song. 

### We will filter out results based on session_id and item_in_session 

### Keys will be session id and item in session as a composite key to fulfill the selection criteria. We will use session id as partition key and item_in_session as clustering column

In [9]:
query = "CREATE TABLE IF NOT EXISTS T_MUSIC "
query = query + "(artist text, first_name text, gender text, item_in_session int, last_name text, \
length float, level text, location text, session_id int,song text,user_id int, \
PRIMARY KEY(session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT statements into the `query` variable
        query = "INSERT INTO T_MUSIC(artist, first_name, gender,item_in_session,last_name, length, \
        level, location, session_id,song,user_id)"
        query = query + "values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0],line[1],line[2],int(line[3]),line[4],float(line[5]),line[6],line[7],
                                int(line[8]),line[9],int(line[10])))
    print(f'Data inserted successfully to T_MUSIC')

Data inserted successfully to T_MUSIC


#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
##SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [12]:
## SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2:

### Extract the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
### for userid = 10, sessionid = 182

In [22]:
query2 = "select artist,song,first_name,last_name from T_MUSIC_BY_USER where user_id = 10 and session_id = 182"

### In this query, we will select columns artist, song title, first name, last name 

### We will filter out results based on user_id and session_id

### Composite key will have user_id, session id and item_in_session as Keys.

### We will use user_id and session id as partition key and item_in_session as clustering column , as we need sorting  based on item_in_session column

In [15]:
query = "CREATE TABLE IF NOT EXISTS T_MUSIC_BY_USER"
query = query + "(artist text, first_name text, gender text, item_in_session int, last_name text, \
length float, level text, location text, session_id int,song text,user_id int, \
PRIMARY KEY((user_id, session_id) , item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [19]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO T_MUSIC_BY_USER(artist, first_name, gender,item_in_session,last_name, length, \
        level, location, session_id,song,user_id)"
        query = query + "values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0],line[1],line[2],int(line[3]),line[4],float(line[5]),line[6],line[7],
                                int(line[8]),line[9],int(line[10])))
    print(f'Data inserted successfully to T_MUSIC_BY_USER')

Data inserted successfully to T_MUSIC_BY_USER


In [23]:
##Add in the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.first_name, row.item_in_session)

Down To The Bone Keep On Keepin' On Sylvie 0
Three Drives Greece 2000 Sylvie 1
Sebastien Tellier Kilometer Sylvie 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie 3


## Query 3:

### Let's select every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [24]:
query3 = "select first_name,last_name from T_USER_BY_SONG where song = 'All Hands Against His Own'"                    

### In this query, we will select columns first name, last name 

### We will filter out results based on song title

### Composite key will have song and user_id as Keys.

### We will use song id as partition key and user_id as clustering column

In [25]:
query = "CREATE TABLE IF NOT EXISTS T_USER_BY_SONG"
query = query + "(artist text, first_name text, gender text, item_in_session int, last_name text, \
length float, level text, location text, session_id int,song text,user_id int, \
PRIMARY KEY(song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [26]:
# Populate tables from the CSV
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO T_USER_BY_SONG(artist, first_name, gender,item_in_session,last_name, length, \
        level, location, session_id,song,user_id)"
        query = query + "values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[0],line[1],line[2],int(line[3]),line[4],float(line[5]),line[6],line[7],
                                int(line[8]),line[9],int(line[10])))
    print(f'Data inserted successfully to T_USER_BY_SONG')

Data inserted successfully to T_USER_BY_SONG


In [27]:
## SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name,row.last_name)

Jacqueline Lynch
Sara Johnson
Tegan Levine


### As a next step, we will drop the tables before closing out the sessions

In [ ]:
## Drop the table before closing out the sessions
query = "DROP TABLE T_MUSIC "
query1 = "DROP TABLE T_MUSIC_BY_USER"
query2 = "DROP TABLE T_USER_BY_SONG"
try:
    session.execute(query)
except Exception as e:
    print(e)
try:
    session.execute(query1)
except Exception as e:
    print(e)
try:
    session.execute(query2)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [29]:
session.shutdown()
cluster.shutdown()